In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import qmc
from os import path
from self_driving_lab_demo.utils.plotting import plot_and_save

In [2]:
bounds = {"x1": [0, 1], "x2": [0, 1]}
num_samples = 10

## Uninformed Sampling Methods

I.e. sampling methods that do not incorporate information about the objective function
to be optimized.

### Grid Samples

In [3]:
from sklearn.model_selection import ParameterGrid

def get_grid_samples(bounds, num_samples = 10):
    param_grid = {}
    num_pts_per_dim = round(num_samples ** (1 / len(bounds)))
    for name, bnd in bounds.items():
        param_grid[name] = np.linspace(bnd[0], bnd[1], num=num_pts_per_dim)
    print(num_pts_per_dim)
    return pd.DataFrame(list(ParameterGrid(param_grid)))

grid_samples = get_grid_samples(bounds, num_samples=num_samples)
grid_samples

3


,x1,x2
0,0.0,0.0
1,0.0,0.5
2,0.0,1.0
3,0.5,0.0
4,0.5,0.5
5,0.5,1.0
6,1.0,0.0
7,1.0,0.5
8,1.0,1.0


In [4]:
grid_fig = px.scatter(grid_samples, x="x1", y="x2", width=400, height=400)
grid_fig

### Random Samples

In [5]:
from numpy.random import default_rng

def get_random_samples(bounds, num_samples=9, seed=None):
    rng = default_rng(seed)
    samples = {}
    for parameter, bound in bounds.items():
        samples[parameter] = rng.uniform(bound[0], bound[1], num_samples)
    return pd.DataFrame(samples)

random_samples = get_random_samples(bounds, seed=0)
random_samples

,x1,x2
0,0.636962,0.935072
1,0.269787,0.815854
2,0.040974,0.002739
3,0.016528,0.857404
4,0.813270,0.033586
5,0.912756,0.729655
6,0.606636,0.175656
7,0.729497,0.863179
8,0.543625,0.541461


In [6]:
random_fig = px.scatter(random_samples, x="x1", y="x2", width=400, height=400)
random_fig

### Latin Hypercube Samples

In [7]:
def get_latin_hypercube_samples(bounds, num_samples=10, seed=None):
    sampler = qmc.LatinHypercube(d=len(bounds), optimization="random-cd", seed=seed)
    samples = sampler.random(num_samples)
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    samples = qmc.scale(samples, l_bounds, u_bounds)
    return pd.DataFrame(samples, columns=list(bounds.keys()))

latin_hypercube_samples = get_latin_hypercube_samples(bounds, seed=0)
latin_hypercube_samples

,x1,x2
0,0.245638,0.173021
1,0.436304,0.298347
2,0.927034,0.708724
3,0.114260,0.827050
4,0.718673,0.006493
5,0.013682,0.499726
6,0.595903,0.996641
7,0.639336,0.582434
8,0.318415,0.645854
9,0.870029,0.357731


In [8]:
latin_hypercube_fig = px.scatter(
    latin_hypercube_samples, x="x1", y="x2", width=400, height=400
)
latin_hypercube_fig


### Sobol Samples

In [9]:
from scipy.stats.qmc import Sobol

def get_sobol_samples(bounds, num_samples=10, seed=None):
    sampler = Sobol(len(bounds), seed=seed)
    samples = sampler.random(num_samples)
    
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    samples = qmc.scale(samples, l_bounds, u_bounds)
    
    return pd.DataFrame(samples, columns=list(bounds.keys()))

sobol_samples = get_sobol_samples(bounds, num_samples=num_samples, seed=0)
sobol_samples

C:\Users\sterg\AppData\Local\Temp\ipykernel_33080\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



,x1,x2
0,0.850585,0.931366
1,0.451565,0.166937
2,0.248736,0.591645
3,0.584153,0.326728
4,0.663688,0.711389
5,0.014668,0.448486
6,0.312342,0.808678
7,0.897760,0.046263
8,0.987552,0.509826
9,0.339692,0.274682


In [10]:
sobol_fig = px.scatter(sobol_samples, x="x1", y="x2", width=400, height=400)
sobol_fig

### Comparison between sampling methods

In [11]:
sampling_fns = dict(
    grid=get_grid_samples,
    random=get_random_samples,
    latin_hypercube=get_latin_hypercube_samples,
    sobol=get_sobol_samples,
)

sample_nums = [5, 10, 50, 100]
sample_nums.reverse()
        
sample_dfs = []
for name, sampling_fn in sampling_fns.items():
    for num_samples in sample_nums:
        sample_df = sampling_fn(bounds, num_samples)
        sample_df["name"] = name
        sample_df["num_samples"] = num_samples
        sample_dfs.append(sample_df)

compare_df = pd.concat(sample_dfs, axis=0)

10
7
3
2


C:\Users\sterg\AppData\Local\Temp\ipykernel_33080\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_33080\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_33080\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.

C:\Users\sterg\AppData\Local\Temp\ipykernel_33080\2746881076.py:5: UserWarning:

The balance properties of Sobol' points require n to be a power of 2.



In [12]:
compare_df

,x1,x2,name,num_samples
0,0.000000,0.000000,grid,100
1,0.000000,0.111111,grid,100
2,0.000000,0.222222,grid,100
3,0.000000,0.333333,grid,100
4,0.000000,0.444444,grid,100
...,...,...,...,...
0,0.417135,0.732311,sobol,5
1,0.725262,0.024790,sobol,5
2,0.908593,0.946108,sobol,5
3,0.234285,0.308906,sobol,5


In [17]:
fig = px.scatter(
    compare_df,
    x="x1",
    y="x2",
    facet_row="num_samples",
    facet_col="name",
    width=800,
    height=800,
)
plot_and_save(
    "traditional-doe-compare",
    fig,
    show=True,
    mpl_kwargs=dict(width_inches=7.5, height_inches=8.0),
)


## Code Graveyard

In [ ]:
# from ax.modelbridge.factory import get_sobol
# from ax.service.ax_client import AxClient

# def get_sobol_samples(bounds, num_samples=10):
#     parameters = [
#             {
#                 "name": "x1",
#                 "type": "range",
#                 "bounds": bounds["x1"],
#             },
#             {
#                 "name": "x2",
#                 "type": "range",
#                 "bounds": bounds["x2"],
#             },
#         ]

#     client = AxClient()
#     client.create_experiment(
#         name="experiment",
#         parameters=parameters,  # type: ignore
#     )
        
#     m = get_sobol(client.experiment.search_space)
#     gr = m.gen(n=num_samples)
#     gr

In [ ]:
# compare_samples = {}
# for name, sampling_fn in sampling_fns.items():
#     compare_samples[name] = {}
#     for num_samples in sample_nums:
#         compare_samples[name][num_samples] = sampling_fn(bounds, num_samples)
        
# df = pd.DataFrame(compare_samples)
# df.index.name = "num_samples"
# df